In [33]:
import cv2
import numpy as np
from keras.datasets import mnist
from keras import utils
import tensorflow as tf
from PIL import Image
import os



train_path = './AIOT/'    
train_txt = './label.txt'  
x_train_savepath = './mnist_x_train.npy'   
y_train_savepath = './mnist_y_train.npy'   

test_path = './AIOT/'     
test_txt = './label.txt'   
x_test_savepath = './mnist_x_test.npy'     
y_test_savepath = './mnist_y_test.npy'     


def generateds(path, txt):      
    f = open(txt, 'r')          
    contents = f.readlines()    
    f.close()                   
    x, y_ = [], []              
    for content in contents:   
        value = content.split()           
        img_path = path + value[0]        
        img = Image.open(img_path)        
        img = np.array(img.convert('L'))  
        img = img / 255.                  
        x.append(img)                     
        y_.append(value[1])               
    

    x = np.array(x)          
    y_ = np.array(y_)         
    y_ = y_.astype(np.int64)  
    return x, y_             

if os.path.exists(x_train_savepath) and os.path.exists(y_train_savepath) and os.path.exists(
        x_test_savepath) and os.path.exists(y_test_savepath):  
    print('-------------Load Datasets-----------------')
    x_train_save = np.load(x_train_savepath)
    y_train = np.load(y_train_savepath)
    x_test_save = np.load(x_test_savepath)
    y_test = np.load(y_test_savepath)
    x_train = np.reshape(x_train_save, (len(x_train_save), 28, 28))  
    x_test = np.reshape(x_test_save, (len(x_test_save), 28, 28))    
else:  
    print('-------------Generate Datasets-----------------')
    x_train, y_train = generateds(train_path, train_txt)
    x_test, y_test = generateds(test_path, test_txt)
    print('-------------Save Datasets-----------------')
    x_train_save = np.reshape(x_train, (len(x_train), -1))
    x_test_save = np.reshape(x_test, (len(x_test), -1))
    np.save(x_train_savepath, x_train_save)
    np.save(y_train_savepath, y_train)
    np.save(x_test_savepath, x_test_save)
    np.save(y_test_savepath, y_test)










# (x_train, y_train), (x_test, y_test) = mnist.load_data()  # 載入訓練集

# 訓練集資料
print(x_train.shape[0])
x_train = x_train.reshape(x_train.shape[0],-1)  # 轉換資料形狀
x_train = x_train.astype('float32')/255         # 轉換資料型別
y_train = y_train.astype(np.float32)
print(type(x_train))
print(type(y_train))

# 測試集資料
x_test = x_test.reshape(x_test.shape[0],-1)     # 轉換資料形狀
x_test = x_test.astype('float32')/255           # 轉換資料型別
y_test = y_test.astype(np.float32)

knn=cv2.ml.KNearest_create()                    # 建立 KNN 訓練方法
knn.setDefaultK(5)                              # 參數設定
knn.setIsClassifier(True)

print('training...')
knn.train(x_train, cv2.ml.ROW_SAMPLE, y_train)  # 開始訓練
knn.save('mnist_knn_test.xml')                       # 儲存訓練模型
print('ok')

print('testing...')
test_pre = knn.predict(x_test)                  # 讀取測試集並進行辨識
test_ret = test_pre[1]
test_ret = test_ret.reshape(-1,)
test_sum = (test_ret == y_test)
acc = test_sum.mean()                           # 得到準確率
print(acc)

-------------Generate Datasets-----------------
-------------Save Datasets-----------------
241
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
training...
ok
testing...
0.9253112033195021
